# The Stanford Sentiment Treebank 
The Stanford Sentiment Treebank consists of sentences from movie reviews and human annotations of their sentiment. The task is to predict the sentiment of a given sentence. We use the two-way (positive/negative) class split, and use only sentence-level labels.

In [1]:
from IPython.display import display, Markdown
with open('../../doc/env_variables_setup.md', 'r') as fh:
    content = fh.read()
display(Markdown(content))

Environment variables that need to be defined:   
`export DIR_PROJ=your_path_git_repository`  
`export PYTHONPATH=$DIR_PROJ/src`  
`export PATH_TENSORBOARD=your_path_tensorboard`  
`export PATH_DATASETS=your_path_datasets`  
`export PROJECT_ID=your_gcp_project_id`  
`export BUCKET_NAME=your_gcp_gs_bucket_name`  
`export BUCKET_TRANSLATION_NAME=your_gcp_gs_bucket_translation_name`  
`export BUCKET_STAGING_NAME=your_gcp_gs_bucket_staging_name` 
`export REGION=your_region`  
`export PATH_SAVE_MODEL=your_path_to_save_model`  
`export CLOUDSDK_PYTHON=your_path/conda-env/env_gcp_sdk/bin/python`  
`export CLOUDSDK_GSUTIL_PYTHON=your_path/conda-env/env_gcp_sdk/bin/python`  

- Use local Jupyter Lab 
    - you need to have the `jupyter-notebook` Anaconda python environment created [link](local_jupyter_lab_installation.md) 
    - you need to have the `jupyter-notebook` Anaconda python environment activated [link](local_jupyter_lab_installation.md) 
    - then define the environment variables above (copy and paste) 
    - you need to have the `env_multilingual_class` Anaconda python environment created [link](local_jupyter_lab_installation.md)  
    - start Jupyter Lab:  `jupyter lab` 
    - open a Jupyter Lab notebook from `notebook/` 
     - clone this repositiory: `git clone https://github.com/tarrade/proj_multilingual_text_classification.git`
    - choose the proper Anaconda python environment:  `Python [conda env:env_multilingual_class]` [link](conda_env.md) 
    - clone this repositiory: `git clone https://github.com/tarrade/proj_multilingual_text_classification.git`


- Use GCP Jupyter Lab 
    - Go on GCP
    - open a Cloud Shell
    - `ssh-keygen -t rsa -b 4096 -C firstName_lastName`
    - `cp .ssh/id_rsa.pub .`
    - use Cloud Editor to edit this file `id_rsa.pub` and copy the full content
    - Go on Compute Engine -> Metadata
    - Click SSH Keys
    - Click Edit
    - Click + Add item, copy the content of `id_rsa.pub`
    - You should see firstName_lastName of the left
    - Click Save
    - you need to start a AI Platform instance 
    - open a Jupyter Lab terminal and got to `/home/gcp_user_name/`
    - clone this repositiory: `git clone https://github.com/tarrade/proj_multilingual_text_classification.git`
    - then `cd proj_multilingual_text_classification/`
    - create the Anacond Python environment `conda env create -f env/environment.yml`
    - create a file `config.sh` in `/home` with the following information: 
    ```
    #!/bin/bash
    
    echo "applying some configuration ..."
    git config --global user.email user_email
    git config --global user.name user_name
    git config --global credential.helper store
        
    # Add here the enviroment variables from above below
    # [EDIT ME]
    export DIR_PROJ=your_path_git_repository
    export PYTHONPATH=$DIR_PROJ/src
  
    cd /home/gcp_user_name/
    
    conda activate env_multilingual_class

    export PS1='\[\e[91m\]\u@:\[\e[32m\]\w\[\e[0m\]$'
    ```
    - Got to AI Platform Notebook, select your instance and click "Reset".
    - Wait and reshreh you Web browser with the Notebook


## Import Packages

In [69]:
import tensorflow as tf
from transformers import (
    BertConfig,
    BertTokenizer,
    XLMRobertaTokenizer,
    TFBertModel,
    TFXLMRobertaModel,
)
import os
from datetime import datetime
import tensorflow_datasets
from tensorboard import notebook
import math
#from google.cloud import storage
from googleapiclient import discovery
from googleapiclient import errors
import logging
import json

## Check configuration

In [70]:
print(tf.version.GIT_VERSION, tf.version.VERSION)

v2.2.0-rc4-8-g2b96f3662b 2.2.0


In [71]:
print(tf.keras.__version__)

2.3.0-tf


In [72]:
gpus = tf.config.list_physical_devices('GPU')
if len(gpus)>0:
    for gpu in gpus:
        print('Name:', gpu.name, '  Type:', gpu.device_type)
else:
    print('No GPU available !!!!')

No GPU available !!!!


## Define Paths

In [73]:
try:
    data_dir=os.environ['PATH_DATASETS']
except KeyError:
    print('missing PATH_DATASETS')
try:   
    tensorboard_dir=os.environ['PATH_TENSORBOARD']
except KeyError:
    print('missing PATH_TENSORBOARD')
try:   
    savemodel_dir=os.environ['PATH_SAVE_MODEL']
except KeyError:
    print('missing PATH_SAVE_MODEL')

# Import local packages

In [74]:
import utils.model_utils as mu

In [75]:
import importlib
importlib.reload(mu);

## Train the model on AI Platform Training (for production)

In [82]:
project_name = os.environ['PROJECT_ID']
project_id = 'projects/{}'.format(project_name)
ai_platform_training = discovery.build('ml', 'v1')

/Users/tarrade/anaconda-release/conda-env/env_multilingual_class/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [112]:
# choose the model
model_name = 'tf_bert_classification'

# variable used to build some variable's name
type_production = 'test' #'test', 'production'
hardware = 'cpu' #'cpu', 'gpu', 'tpu'
owner = os.environ['OWNER']
tier = 'basic' #'basic', 'custom'
hp_tuning= True
verbosity = 'INFO'
profiling = False

# overwrite parameter for testing logging
test_logging = True

print(' modifying Tensorflow env variable')
# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed
with open(os.environ['DIR_PROJ']+'/utils/env_variables.json', 'r') as outfile:
    env_var = json.load(outfile)
if verbosity == 'DEBUG' or  verbosity == 'VERBOSE' or verbosity == 'INFO':
    env_var['TF_CPP_MIN_LOG_LEVEL'] = 0
    env_var['TF_CPP_MIN_VLOG_LEVEL'] = 0
elif verbosity == 'WARNING':
    env_var['TF_CPP_MIN_LOG_LEVEL'] = 1
    env_var['TF_CPP_MIN_VLOG_LEVEL'] = 1
elif verbosity == 'ERROR':
    env_var['TF_CPP_MIN_LOG_LEVEL'] = 2
    env_var['TF_CPP_MIN_VLOG_LEVEL'] = 2
else:
    env_var['TF_CPP_MIN_LOG_LEVEL'] = 3
    env_var['TF_CPP_MIN_VLOG_LEVEL'] = 3
print("env_var['TF_CPP_MIN_LOG_LEVEL']=", env_var['TF_CPP_MIN_LOG_LEVEL'])
print("env_var['TF_CPP_MIN_VLOG_LEVEL']=", env_var['TF_CPP_MIN_VLOG_LEVEL'])
data={}
data['TF_CPP_MIN_LOG_LEVEL'] = env_var['TF_CPP_MIN_LOG_LEVEL']
data['TF_CPP_MIN_VLOG_LEVEL'] = env_var['TF_CPP_MIN_VLOG_LEVEL']
with open(os.environ['DIR_PROJ']+'/utils/env_variables.json', 'w') as outfile:
    json.dump(data, outfile)

# define parameters for ai platform training
package_gcs = mu.create_module_tar_archive(model_name)

timestamp = datetime.now().strftime("%Y_%m_%d_%H%M%S")
if hp_tuning:
    job_name = model_name+'_hp_tuning_'+hardware+'_'+timestamp
else:
    job_name = model_name+'_'+hardware+'_'+timestamp
module_name = 'model.'+model_name+'.task'
if tier=='basic' and hardware=='cpu':
    # CPU
    region = 'europe-west1'
    
elif tier=='basic' and hardware=='gpu':
    # GPU
    region = 'europe-west1'
    
elif tier=='custom' and hardware=='gpu':
    # Custom GPU
    region = 'europe-west4'
    
elif tier=='basic' and hardware=='tpu':
    # TPU
    #region = 'us-central1'
    region = 'europe-west4' # No zone in region europe-west4 has accelerators of all requested types
    #region = 'europe-west6' # The request for 8 TPU_V2 accelerators exceeds the allowed maximum of 0 K80, 0 P100, 0 P4, 0 T4, 0 TPU_V2, 0 TPU_V2_POD, 0 TPU_V3, 0 TPU_V3_POD, 0 V100
    #region = 'europe-west2'  # No zone in region europe-west2 has accelerators of all requested types

elif tier=='custom' and hardware=='tpu':
    # TPU
    #region = 'us-central1'
    region = 'europe-west4'
    #region = 'europe-west6'
    #region = 'europe-west2'

else:
    # Default
    region = 'europe-west1'

# define parameters for training of the model
if type_production=='production':
    # reading metadata
    _, info = tensorflow_datasets.load(name='glue/sst2',
                                       data_dir=data_dir,
                                       with_info=True)
    # define parameters
    epochs = 2 
    batch_size_train = 32
    #batch_size_test = 32
    batch_size_eval = 64  
    
    # Maxium length, becarefull BERT max length is 512!
    max_length = 128

    # extract parameters
    size_train_dataset=info.splits['train'].num_examples
    #size_test_dataset=info.splits['test'].num_examples
    size_valid_dataset=info.splits['validation'].num_examples

    # computer parameter
    steps_per_epoch_train = math.ceil(size_train_dataset/batch_size_train)
    #steps_per_epoch_test = math.ceil(size_test_dataset/batch_size_test)
    steps_per_epoch_eval = math.ceil(size_valid_dataset/batch_size_eval)

    #print('Dataset size:          {:6}/{:6}/{:6}'.format(size_train_dataset, size_test_dataset, size_valid_dataset))
    #print('Batch size:            {:6}/{:6}/{:6}'.format(batch_size_train, batch_size_test, batch_size_eval))
    #print('Step per epoch:        {:6}/{:6}/{:6}'.format(steps_per_epoch_train, steps_per_epoch_test, steps_per_epoch_eval))
    #print('Total number of batch: {:6}/{:6}/{:6}'.format(steps_per_epoch_train*(epochs+1), steps_per_epoch_test*(epochs+1), steps_per_epoch_eval*1))
    print('Number of epoch:        {:6}'.format(epochs))
    print('Batch size:            {:6}/{:6}'.format(batch_size_train, batch_size_eval))
    print('Step per epoch:        {:6}/{:6}'.format(steps_per_epoch_train, steps_per_epoch_eval))

else:
    if hardware=='tpu':
        epochs = 1 
        steps_per_epoch_train = 5 
        batch_size_train = 32 
        steps_per_epoch_eval = 1 
        batch_size_eval = 64
    else:
        epochs = 1 
        steps_per_epoch_train = 5 
        batch_size_train = 32 
        steps_per_epoch_eval = 1 
        batch_size_eval = 64
        
steps=epochs*steps_per_epoch_train
if steps<=5:
    n_steps_history=4
elif steps>=5 and steps<1000:
    n_steps_history=10
    print('be carefull with profiling between step: 10-20')
else:
    n_steps_history=int(steps/100)
    print('be carefull with profiling between step: 10-20')
print('will compute accuracy on the test set every {} step so {} time'.format(n_steps_history, int(steps/n_steps_history)))

if profiling:
    print(' profilin ...')
    steps_per_epoch_train = 100 
    n_steps_history=25

input_eval_tfrecords = 'gs://'+os.environ['BUCKET_NAME']+'/tfrecord/sst2/bert-base-multilingual-uncased/valid' #'gs://public-test-data-gs/valid'
input_train_tfrecords = 'gs://'+os.environ['BUCKET_NAME']+'/tfrecord/sst2/bert-base-multilingual-uncased/train' #'gs://public-test-data-gs/train'
if hp_tuning:
    output_dir = 'gs://'+os.environ['BUCKET_NAME']+'/training_model_gcp/'+model_name+'_hp_tuning_'+hardware+'_'+timestamp
else:
    output_dir = 'gs://'+os.environ['BUCKET_NAME']+'/training_model_gcp/'+model_name+'_'+hardware+'_'+timestamp
pretrained_model_dir = 'gs://'+os.environ['BUCKET_NAME']+'/pretrained_model/bert-base-multilingual-uncased'
epsilon = 1.7788921050163616e-06
learning_rate= 0.0007763625134788308

# bulding training_inputs
parameters =  ['--epochs', str(epochs),
               '--steps_per_epoch_train', str(steps_per_epoch_train),
               '--batch_size_train', str(batch_size_train),
               '--steps_per_epoch_eval', str(steps_per_epoch_eval),
               '--n_steps_history', str(n_steps_history),
               '--batch_size_eval', str(batch_size_eval),
               '--input_eval_tfrecords', input_eval_tfrecords ,
               '--input_train_tfrecords', input_train_tfrecords,
               '--output_dir', output_dir,
               '--pretrained_model_dir', pretrained_model_dir,
               '--verbosity_level', verbosity,
               '--epsilon', str(epsilon),
               '--learning_rate', str(learning_rate)]
if hardware=='tpu':
    parameters.append('--use_tpu')
    parameters.append('True')

training_inputs = {
    'packageUris': [package_gcs],
    'pythonModule': module_name,
    'args': parameters,
    'region': region,
    'runtimeVersion': '2.1',
    'pythonVersion': '3.7',
}

if tier=='basic' and hardware=='cpu':
    # CPU
    training_inputs['scaleTier'] = 'BASIC'
    
elif tier=='basic' and hardware=='gpu':
    # GPU
    training_inputs['scaleTier'] = 'BASIC_GPU'
    
elif tier=='custom' and hardware=='gpu':
    # Custom GPU
    training_inputs['scaleTier'] = 'CUSTOM'
    training_inputs['masterType'] = 'n1-standard-8'
    accelerator_master = {'acceleratorConfig': {
        'count': '1',
        'type': 'NVIDIA_TESLA_V100'}
    }
    training_inputs['masterConfig'] = accelerator_master

    
elif tier=='basic' and hardware=='tpu':
    # TPU
    training_inputs['scaleTier'] = 'BASIC_TPU'
    
elif tier=='custom' and hardware=='tpu':
    # Custom TPU
    training_inputs['scaleTier'] = 'CUSTOM'
    training_inputs['masterType'] = 'n1-highcpu-16'
    training_inputs['workerType'] = 'cloud_tpu'
    training_inputs['workerCount'] = '1'
    accelerator_master = {'acceleratorConfig': {
        'count': '8',
        'type': 'TPU_V3'}
    }
    training_inputs['workerConfig'] = accelerator_master

else:
    # Default
    training_inputs['scaleTier'] = 'BASIC'

# add hyperparameter tuning to the job config.
if hp_tuning:
    hyperparams = {
        'algorithm': 'ALGORITHM_UNSPECIFIED',
        'goal': 'MAXIMIZE',
        'hyperparameterMetricTag': 'metric1',
        'maxTrials': 3,
        'maxParallelTrials': 2,
        'maxFailedTrials': 1,
        'enableTrialEarlyStopping': True,
        'hyperparameterMetricTag': 'metric_accuracy_train_epoch',
        'params': []}

    hyperparams['params'].append({
        'parameterName':'learning_rate',
        'type':'DOUBLE',
        'minValue': 1.0e-8,
        'maxValue': 1.0,
        'scaleType': 'UNIT_LOG_SCALE'})
    
    hyperparams['params'].append({
        'parameterName':'epsilon',
        'type':'DOUBLE',
        'minValue': 1.0e-9,
        'maxValue': 1.0,
        'scaleType': 'UNIT_LOG_SCALE'})

    # Add hyperparameter specification to the training inputs dictionary.
    training_inputs['hyperparameters'] = hyperparams
    
# building job_spec
labels = {'accelerator': hardware,
          'type': type_production,
          'owner': owner}

job_spec = {'jobId': job_name, 
            'labels': labels, 
            'trainingInput': training_inputs}

if test_logging:
    # test
    # variable used to build some variable's name
    owner = os.environ['OWNER']
    tier = 'basic' 
    verbosity = 'INFO'

    # define parameters for ai platform training
    package_gcs = package_gcs

    job_name = 'debug_test_'+datetime.now().strftime("%Y_%m_%d_%H%M%S")

    module_name = 'model.test.task'

    region = 'europe-west1'

    # bulding training_inputs
    parameters =  ['--verbosity_level', verbosity]

    training_inputs = {
        'packageUris': [package_gcs],
        'pythonModule': module_name,
        'args': parameters,
        'region': region,
        'runtimeVersion': '2.1',
        'pythonVersion': '3.7',
    }

    training_inputs['scaleTier'] = 'BASIC'

    # building job_spec
    labels = {'accelerator': 'cpu',
              'type': 'debug',
              'owner': owner}

    job_spec = {'jobId': job_name, 
                'labels': labels, 
                'trainingInput': training_inputs}

 modifying Tensorflow env variable
env_var['TF_CPP_MIN_LOG_LEVEL']= 0
env_var['TF_CPP_MIN_VLOG_LEVEL']= 0


running sdist
running egg_info
writing bert_model.egg-info/PKG-INFO
writing dependency_links to bert_model.egg-info/dependency_links.txt
writing requirements to bert_model.egg-info/requires.txt
writing top-level names to bert_model.egg-info/top_level.txt
reading manifest file 'bert_model.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'bert_model.egg-info/SOURCES.txt'
running check
creating bert_model-0.1
creating bert_model-0.1/analysis
creating bert_model-0.1/bert_model.egg-info
creating bert_model-0.1/model
creating bert_model-0.1/model/sklearn_naive_bayes
creating bert_model-0.1/model/test
creating bert_model-0.1/model/tf_bert_classification
creating bert_model-0.1/model/tf_custom_bert_classification
creating bert_model-0.1/preprocessing
creating bert_model-0.1/utils
copying files to bert_model-0.1...
copying MANIFEST.in -> bert_model-0.1
c

/Users/tarrade/anaconda-release/conda-env/env_multilingual_class/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/tarrade/anaconda-release/conda-env/env_multilingual_class/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" er

will compute accuracy on the test set every 4 step so 1 time


In [113]:
training_inputs

{'packageUris': ['gs://ai-platform-training-package-staging/tf_bert_classification_2020_06_11_103250/bert_model-0.1.tar.gz'],
 'pythonModule': 'model.test.task',
 'args': ['--verbosity_level', 'INFO'],
 'region': 'europe-west1',
 'runtimeVersion': '2.1',
 'pythonVersion': '3.7',
 'scaleTier': 'BASIC'}

In [114]:
# submit the training job
request = ai_platform_training.projects().jobs().create(body=job_spec,
                                                        parent=project_id)
try:
    response = request.execute()
    print('Job status for {}:'.format(response['jobId']))
    print('    state : {}'.format(response['state']))
    print('    createTime: {}'.format(response['createTime']))

except errors.HttpError as err:
    # For this example, just send some text to the logs.
    # You need to import logging for this to work.
    logging.error('There was an error creating the training job.'
                  ' Check the details:')
    logging.error(err._get_reason())

Job status for debug_test_2020_06_11_103252:
    state : QUEUED
    createTime: 2020-06-11T08:32:53Z


In [80]:
# if you wnat to specify a specif job ID
#job_name = 'tf_bert_classification_2020_05_16_193551'
jobId = 'projects/{}/jobs/{}'.format(project_name, job_name)
request = ai_platform_training.projects().jobs().get(name=jobId)
response = None

try:
    response = request.execute()
    print('Job status for {}:'.format(response['jobId']))
    print('    state : {}'.format(response['state']))
    if 'trainingOutput' in response.keys():
        if 'trials' in response['trainingOutput'].keys():
            for sub_job in response['trainingOutput']['trials']:
                print('    trials : {}'.format(sub_job))
    if 'consumedMLUnits' in response.keys():
        print('    consumedMLUnits : {}'.format(response['trainingOutput']['consumedMLUnits']))
    if 'errorMessage' in response.keys():
        print('    errorMessage : {}'.format(response['errorMessage']))
    
except errors.HttpError as err:
    logging.error('There was an error getting the logs.'
                  ' Check the details:')
    logging.error(err._get_reason())

Job status for tf_bert_classification_hp_tuning_cpu_2020_06_10_113245:
    state : SUCCEEDED
    trials : {'trialId': '1', 'hyperparameters': {'learning_rate': '0.0001000000000000001', 'epsilon': '3.1622776601683782e-05'}, 'startTime': '2020-06-10T09:33:06.656329913Z', 'endTime': '2020-06-10T09:50:00Z', 'state': 'FAILED'}
    trials : {'trialId': '2', 'hyperparameters': {'learning_rate': '1.318427816398073e-05', 'epsilon': '3.2360572457522847e-06'}, 'startTime': '2020-06-10T09:33:06.656458859Z', 'endTime': '2020-06-10T09:48:29Z', 'state': 'FAILED'}
    trials : {'trialId': '3', 'hyperparameters': {'learning_rate': '1.4459909039789456e-06', 'epsilon': '2.6917926744580865e-07'}, 'startTime': '2020-06-10T09:50:00.097393570Z', 'endTime': '2020-06-10T10:05:52Z', 'state': 'FAILED'}


In [81]:
# how to stream logs
# --stream-logs

# TensorBoard for job running on GCP

In [14]:
# View open TensorBoard instance
#notebook.list() 

In [15]:
# View pid
#!ps -ef|grep tensorboard

In [16]:
# Killed Tensorboard process by using pid
#!kill -9 pid

In [40]:
%load_ext tensorboard
#%reload_ext tensorboard
%tensorboard  --logdir {output_dir+'/tensorboard'} \
              #--host 0.0.0.0 \
              #--port 6006 \
              #--debugger_port 6006

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [191]:
%load_ext tensorboard
#%reload_ext tensorboard
%tensorboard  --logdir {output_dir+'/hparams_tuning'} \
              #--host 0.0.0.0 \
              #--port 6006 \
              #--debugger_port 6006

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
